In [18]:
from sliderule import icesat2
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import contextily as cx
import os
from os.path import join, exists, basename
import warnings
import pickle 
from glob import glob
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
polys = {}
for fp in glob(join('/Users/zachkeskinen/Documents/isce_sat2/data/uavsar_shape_files', '*.shp')):
    name = basename(fp).split('_')[0]
    polys[name] = fp

In [26]:
with open('/Users/zachkeskinen/Documents/isce_sat2/data/poly.')
polys

{'fraser': '/Users/zachkeskinen/Documents/isce_sat2/data/uavsar_shape_files/fraser_23306_21020-026_21021-004_0006d_s01_L090HH_01.cor.grd .shp',
 'rockmt': '/Users/zachkeskinen/Documents/isce_sat2/data/uavsar_shape_files/rockmt_32109_21017-013_21021-001_0012d_s01_L090HH_01.cor.grd .shp',
 'irnton': '/Users/zachkeskinen/Documents/isce_sat2/data/uavsar_shape_files/irnton_01406_21020-028_21022-002_0006d_s01_L090HH_01.cor.grd .shp',
 'donner': '/Users/zachkeskinen/Documents/isce_sat2/data/uavsar_shape_files/donner_03904_20014-003_20016-007_0014d_s01_L090HH_01.cor.grd .shp',
 'lowman': '/Users/zachkeskinen/Documents/isce_sat2/data/uavsar_shape_files/lowman_05208_21019-019_21021-007_0006d_s01_L090HH_01.cor.grd .shp',
 'uticam': '/Users/zachkeskinen/Documents/isce_sat2/data/uavsar_shape_files/uticam_21003_21004-002_21013-003_0034d_s01_L090HH_01.cor.grd .shp',
 'peeler': '/Users/zachkeskinen/Documents/isce_sat2/data/uavsar_shape_files/peeler_31619_20013-003_20017-008_0015d_s01_L090HH_01.cor.grd

In [1]:
warnings.simplefilter(action='ignore', category=FutureWarning)

    
def download_icesat2(poly_dict, directory='/tmp/is2', length=100.0, res=50.0, verbose=False, confidence = False):
    os.makedirs(directory, exist_ok=True)
    icesat2.init("icesat2sliderule.org", verbose=verbose)
    for name, poly in poly_dict.items():
        out_fp = os.path.join(directory,f'{name}_atl06sr.pkl')
        if not os.path.exists(out_fp):
            print(f'Starting on {name}'.center(50, '-'))
            poly = icesat2.toregion(poly)[0]
            result = gpd.GeoDataFrame()
            conf_range = range(2,5)
            if not confidence:
                conf_range = [4]
            for conf in conf_range:
                parms = {"poly": poly,
                "srt": icesat2.SRT_LAND,
                "atl08_class": "atl08_ground",
                "cnf": [conf],
                "len": length,
                "res": res,
                "maxi": 1,
                "t0":'2018-10-01T00:00:00Z',
                "t1":'2022-04-30T00:00:00Z'}
                rsps = icesat2.atl06p(parms)
                rsps['confidence'] = conf
                result = result.append(rsps)
            name = name.replace(' ','')
            name = name.replace(',','')

            with open(out_fp, 'wb') as f:
                pickle.dump(result, f)
        else:
            print(f'{name} already exists. Skipping...')
                        

In [10]:
types = {'confidence':{'res': 50, 'len':100, 'conf':True}, 'sd':{'res': 20, 'len':40, 'conf':False}}
polys = pickle.load(open('data/bounds.pkl', 'rb'))
for k, v in types.items():
    download_icesat2(polys, directory = os.path.join('/tmp/is2', k), res = v['res'], length = v['len'], confidence = v['conf'])

--------------Starting on Utica, MT---------------


CRITICAL:sliderule.sliderule:Unexpected error: ERROR:
CRITICAL:sliderule.sliderule:Unexpected error: ERROR:


KeyboardInterrupt: 